# Swing equation: one generator and one motor/infinite bus 

This script runs down the basics of numerically integrating one generator and one motor obeying a swing equation, i.e.


$$\frac{2H}{\omega_\mathrm{syn}}\frac{\mathrm{d^2 \delta}}{\mathrm{d} t^2} = - \frac{D}{\omega_\mathrm{syn}}\frac{\mathrm{d \delta}}{\mathrm{d} t} + P_\mathrm{mec} - P_\mathrm{e}$$

To write down an 2nd-order ODE to integrate, we have to transform the problem into 2 1st-order ODEs. Introduce a new variable, the angular frequency $\omega=\frac{\mathrm{d \delta}}{\mathrm{d} t}$ and rewrite the swing equation

$$
\begin{align}
\frac{\mathrm{d \delta}}{\mathrm{d} t} &= \omega\\
\frac{2H}{\omega_\mathrm{syn}}\frac{\mathrm{d \omega}}{\mathrm{d} t} &= 
- \frac{D}{\omega_\mathrm{syn}}\omega + P_\mathrm{mec} - P_\mathrm{e}
\end{align}
$$

In [1]:
import numpy as np
from scipy.integrate import odeint

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Make sure to have the `lib.py` in the same folder as where you deploy your notebook

In [3]:
from lib import swing_equation

Now let's consider the power exchange between two units: A motor ($(\delta_\mathrm{g}, \omega_\mathrm{g})$) and a generator ($(\delta_\mathrm{m}, \omega_\mathrm{m})$)

$$
\begin{align}
\frac{\mathrm{d \delta_\mathrm{g}}}{\mathrm{d} t} &= \omega_\mathrm{g}\\
\frac{2H}{\omega_\mathrm{syn}}\frac{\mathrm{d \omega_\mathrm{g}}}{\mathrm{d} t} &= 
- \frac{D}{\omega_\mathrm{syn}}\omega_\mathrm{g} + P_\mathrm{g} - B \sin(\delta_\mathrm{g} - \delta_\mathrm{m})\\
\frac{\mathrm{d \delta_\mathrm{m}}}{\mathrm{d} t} &= \omega_\mathrm{m}\\
\frac{2H}{\omega_\mathrm{syn}}\frac{\mathrm{d \omega_\mathrm{m}}}{\mathrm{d} t} &= 
- \frac{D}{\omega_\mathrm{syn}}\omega_\mathrm{m} + P_\mathrm{m} - B \sin(\delta_\mathrm{m} - \delta_\mathrm{g})\\
\end{align}
$$

For simplicity, consider the damping `D` and H-constant `H` identical for both, simply exchange power $P= P_\mathrm{g} = - + P_\mathrm{m}$, and call it `P`

Define the important constants

In [10]:
P = 0.4

H = 6  # H constant 
D = 4  # damping, usually 2

B = 0.5  # B is 1/X

Write down the right-hand side of the equation as a function, where `y[0]`=$\delta_\mathrm{g}$, `y[1]`=$\omega_\mathrm{g}$, `y[2]`=$\delta_\mathrm{m}$,`y[3]`=$\omega_\mathrm{m}$, and the various constants 

In [11]:
M = (2 * H) / (2* np.pi * 50)
d = D / (2* np.pi * 50)

def f(y, t, d, B, M, P):
    return [y[1],
            ( - d * y[1] + P - B * np.sin(y[0] - y[2]) ) / M, 
            y[3],
            ( - d * y[3] - P - B * np.sin(y[2] - y[0]) ) / M]

Any ODE needs starting conditions. For the case of the swing equation with balanced power, $\omega_\mathrm{g}=\omega_\mathrm{m}=0$, but the angles of the machines are still unknown. We can solve the linear power flow equation to get a good starting condition, i.e.,

$$
\mathbf{P} = - \mathbf{B}\delta
$$

which is solved by

$$
\delta = - \mathbf{B}^\dagger\mathbf{P}
$$

where $\mathbf{B}^\dagger$ is the pseudo-inverse of the admittance matrix $\mathbf{B}$.

In [12]:
initial_phase = -np.dot(np.linalg.pinv(np.array([[-B,B],[B,-B]])), np.array([[P],[-P]]))

initial_state = [initial_phase[0][0], 0, initial_phase[1][0], 0]
#initial_state = [0, 0, 0, 0]

Now define time and integrate the ODE with `scipy`s `odeint`

In [13]:
time = np.linspace(0, 25, 2000)

res = odeint(f, initial_state, time, args=(d, B, M, P)).T  # the .T is just to transpose the results

In [14]:
# Create a subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=[r'Torque angle', 'Angular velocity'], shared_yaxes=False)

fig.add_trace(go.Scatter(x=time, y=res[0, :], mode='lines', name='Generator', line=dict(color='black', width=2), legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=time, y=res[2, :], mode='lines', name='Load', line=dict(color='purple', width=2), legendgroup='1'), row=1, col=1)

fig.update_xaxes(title_text='Time $t$ [s]', row=1, col=1, tickfont=dict(size=16), title_font=dict(size=18))
fig.update_yaxes(title_text='$\delta$ [rad]', range=[-np.pi/2-0.2, np.pi/2+0.2], tickvals=[-np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2], ticktext=[r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$', '0', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$'], row=1, col=1, tickfont=dict(size=16), title_font=dict(size=18))

fig.add_trace(go.Scatter(x=time, y=res[1, :], mode='lines', name='Generator', line=dict(color='black', width=2), legendgroup='2'), row=1, col=2)
fig.add_trace(go.Scatter(x=time, y=res[3, :], mode='lines', name='Load', line=dict(color='purple', width=2), legendgroup='2'), row=1, col=2)

fig.update_xaxes(title_text='Time $t$ [s]', row=1, col=2, tickfont=dict(size=16), title_font=dict(size=18))
fig.update_yaxes(title_text='$\omega$ [rad/s]', range=[-1, 1], row=1, col=2, tickfont=dict(size=16), title_font=dict(size=18))

arrow1 = go.layout.Annotation(dict(x=24.5, y=res[0, -1], xref="x", yref="y",text="", showarrow=True, axref="x", ayref='y', ax=24.5, ay=0, arrowhead=3, arrowwidth=2, arrowcolor='green'))
arrow2 = go.layout.Annotation(dict(x=24.5, y=res[2, -1], xref="x", yref="y",text="", showarrow=True, axref="x", ayref='y', ax=24.5, ay=0, arrowhead=3, arrowwidth=2, arrowcolor='green'))

fig.update_layout(annotations=[arrow1, arrow2])

l = time[np.argmax(res[0, :] - res[2, :])]
x_end = [24.5, 24.5, l, l]
y_end = [res[0, -1], res[2, -1], np.max(res[0, :]), np.min(res[2, :])]
x_start = [24.5, 24.5, l, l]
y_start = [0, 0, 0, 0]

fig.update_layout(annotations=[go.layout.Annotation(dict(x=x0, y=y0, xref="x", yref="y", text="", showarrow=True, axref="x", ayref='y', ax=x1, ay=y1, arrowhead=3, arrowwidth=2, arrowcolor='red',)) for x0,y0,x1,y1 in zip(x_end, y_end, x_start, y_start)])

fig.add_annotation(text=r"{:.2f}º".format((res[0, -1] - res[2, -1])*180/np.pi), x=22, y=0.0, showarrow=False, font=dict(size=18), xref="x", yref="y")
fig.add_annotation(text=r"{:.2f}º".format((np.max(res[0, :] - res[2, :]))*180/np.pi), x=l + 4, y=0.0, showarrow=False, font=dict(size=18), xref="x", yref="y")


fig.update_layout(height=400, width=900, margin=dict(l=0, r=0, t=40, b=0))
fig.show()

This is a little function that includes all the previous steps in just one code

In [15]:
swing_equation(P=0.4, B=0.5, D=4, time=25, initial_state=[0.463, 0, -.463, 0], inf_bus=False);

### Installation and others

This script relies only on `numpy` and `scipy`, which are easily installable with Anaconda

This script was created for the *FYS377 Digital Power Systems*, by *Heidi S. Nygård*. Leonardo Rydin Gorjão. 2023.